## **미분(differentiation)**
- 변수의 움직임에 따른 함수값의 변화를 측정하기 위한 도구로 최적화에서 제일 많이 사용하는 기법이다.
- 미분은 함수 f의 주어진 점에서의 접선의 기울기를 구한다.
- 한 점에서 접선의 기울기를 알면 어느 방향으로 점을 움직여야 함수값이 증가하는지/ 감소하는지 알 수 있다.
- 미분값을 더하면 경사상승법(gradient ascent)이라 하며 함수의 극댓값의 위치를 구할 때 사용한다. 목적함수를 최대화할 때 사용한다.
- 미분값을 빼면 경사하강법(gradient descent)이라 하며 함수의 극솟값의 위치를 구할 때 사용한다. 목적함수를 최소화할 때 사용한다.
- 극값에서는 미분값이 0이므로 더 이상 업데이트가 안 된다. 그러므로 목적함수 최적화가 자동으로 끝난다.

In [2]:
import sympy as sym
from sympy.abc import x

sym.diff(sym.poly(x**2 + 2*x +3), x)

Poly(2*x + 2, x, domain='ZZ')

## **경사하강법**

**알고리즘**<br>
Input : gradient, init, lr, eps / Output : var<br>
gradient : 미분을 계산하는 함수<br>
init : 시작점 / <br>
lr : 학습률 : 미분을 통해 업데이트하는 속도를 조절한다. / <br>
eps : 알고리즘 종료조건<br><hr>
<pre>
    <code>
        var = init
        grad = gradient(var)
        while(abs(grad) > eps) :
            var = var - lr * grad 
            # 종료조건이 성립하기 전까지 미분값을 계속 업데이트한다.
            grad = gradient(var) 
    </code>
</pre>

In [3]:
def func(val):
    fun = sym.poly(x**2 + 2*x + 3)
    return fun.subs(x, val), fun

In [4]:
def func_gradient(fun, val):
    _, function = fun(val)
    diff = sym.diff(function, x)
    return diff.subs(x, val), diff

In [7]:
import numpy as np

In [8]:
def gradient_descent(fun, init_point, lr_rate=1e-2, epsilon=1e-5):
    cnt=0
    val = init_point
    diff, _ = func_gradient(fun, init_point)
    while np.abs(diff) > epsilon:
        val = val - lr_rate*diff
        diff, _ = func_gradient(fun, val)
        cnt += 1
    print('함수 : {}, 연산횟수 : {}, 최소점 : ({}, {}'.format(fun(val)[1], cnt, val, fun(val)[0]))
    
gradient_descent(fun=func, init_point=np.random.uniform(-2, 2))

함수 : Poly(x**2 + 2*x + 3, x, domain='ZZ'), 연산횟수 : 627, 최소점 : (-0.999995075055184, 2.00000000002426


## **변수가 벡터라면**
- 벡터가 입력인 다변수 함수의 경우 편미분(partial differentiation)을 사용한다.
- 각 변수 별로 편미분을 계산한 그레디언트 벡터를 이용하여 경사하강/경사상승법에 사용할 수 있다.
- 앞서 사용한 미분값인 f'(x) 대신 벡터 ∇f를 사용하여 변수 x를 동시ㅔ 업데이터 가능하다. (∇ : nabla)

In [9]:
import sympy as sym
from sympy.abc import x, y

sym.diff(sym.poly(x**2 + 2*x*y + 3) + sym.cos(x+2*y), x)

C:\Users\Huijung\AppData\Local\Temp\ipykernel_13020\1119509424.py:4: SymPyDeprecationWarning: 

Mixing Poly with non-polynomial expressions in binary
operations is deprecated. Either explicitly convert
the non-Poly operand to a Poly with as_poly() or
convert the Poly to an Expr with as_expr().

See https://docs.sympy.org/latest/explanation/active-deprecations.html#deprecated-poly-nonpoly-binary-operations
for details.

This has been deprecated since SymPy version 1.6. It
will be removed in a future version of SymPy.

  sym.diff(sym.poly(x**2 + 2*x*y + 3) + sym.cos(x+2*y), x)


2*x + 2*y - sin(x + 2*y)

**알고리즘**<br>
Input : gradient, init, lr, eps / Output : var<br>
gradient : 그레디언트 벡터를 계산하는 함수<br>
init : 시작점 / <br>
lr : 학습률 : 미분을 통해 업데이트하는 속도를 조절한다. / <br>
eps : 알고리즘 종료조건<br><hr>
<pre>
    <code>
        var = init
        grad = gradient(var)
        # 벡터는 절댓값 대신 노름(norm)을 계산해서 종료조건을 설정한다.
        while(norm(grad) > eps) :
            var = var - lr * grad 
            # 종료조건이 성립하기 전까지 미분값을 계속 업데이트한다.
            grad = gradient(var) 
    </code>
</pre>

In [11]:
# Multivariate Gradient Descent
def eval_(fun, val):
    val_x, val_y = val
    fun_eval = fun.subs(x, val_x).subs(y, val_y)
    return fun_eval

def func_multi(val):
    x_, y_ = val
    func = sym.poly(x**2 + 2*y**2)
    return eval_(func, [x_, y_]), func

def func_gradient(fun, val):
    x_, y_ = val
    _, function = fun(val)
    diff_x = sym.diff(function, x)
    diff_y = sym.diff(function, y)
    grad_vec = np.array([eval_(diff_x, [x_, y_]), eval_(diff_y, [x_, y_])], dtype=float)
    return grad_vec, [diff_x, diff_y]

def gradient_descent(fun, init_point, lr_rate=1e-2, epsilon=1e-5):
    cnt = 0
    val = init_point
    diff, _ = func_gradient(fun, val)
    while np.linalg.norm(diff) > epsilon:
        val = val - lr_rate * diff
        diff, _ = func_gradient(fun, val)
        cnt += 1
        
    print('함수 : {}, 연산횟수 : {}, 최소점 : ({}, {})'.format(fun(val)[1], cnt, val, fun(val)[0]))
    

pt = [np.random.uniform(-2, 2), np.random.uniform(-2, 2)]
gradient_descent(fun=func_multi, init_point=pt)

함수 : Poly(x**2 + 2*y**2, x, y, domain='ZZ'), 연산횟수 : 637, 최소점 : ([4.97010195e-06 1.87200096e-12], 2.47019133770504E-11)
